# Data import

Five type of data:
 - flat files  : typically simple text file that display data as table （相当于普通的文本文档，比如`CSV`文档就是以逗号为分隔的表格）
 - dat from excel
 - databases
 - web
 - statistical software
 
You need to convert each data form into **R data frame**

### Read.csv()

首先要先确定文件**读取的路径**，因为不是所有文件都放在当前的文件夹里（`getwd`)。

方法一： 你可以通过改变当前的路径（`setwd()`)来读取文件：
```r
setwd(newpath)
read.csv("flatfilename.csv",stringsAsFactor = FALSE,...）
```

方法二： 你可以给文件的调用路径赋予到一个`path`：
```r
# 比如调用的text.csv文件在home的my_dir文件夹里（~/my_dir）
path<- file.path("~","my_dir","text.csv",)
read.csv(path,stringsAsFactor = FALSE,...)
```

### read.delim() & read.table

#### read.delim() 
读取以**tab**(`\t`)为分隔符的文本。

#### read.table()
what if....
```r
states2.txt 
 
state/capital/pop_mill/area_sqm 
South Dakota/Pierre/0.853/77116 
New York/Albany/19.746/54555 
Oregon/Salem/3.970/98381 
Vermont/Montpelier/0.627/9616 
Hawaii/Honolulu/1.420/10931 
```
这个时候对于特殊分隔符的就要使用`read.table()`。

read.table()是使用最多的读取函数，因为它能够读取多种格式的文档。并且拥有大量的参数可以设置：

```r
read.table("states2.txt",
           header= TRUE, # 如果为TRUE，则设置第一行为表头
           sep = "/",
           stringsAsFactor = FALSE)
```

参数`col.names`可以用来规定**列名**；

参数`col.class`可以用来规定**列的类型**；这样就可以不管`stringsasfactor`了；
```r
# Previous call to import hotdogs.txt
hotdogs <- read.delim("hotdogs.txt", header = FALSE, col.names = c("type", "calories", "sodium"))

# Display structure of hotdogs
str(hotdogs)

# Edit the colClasses argument to import the data correctly: hotdogs2
hotdogs2 <- read.delim("hotdogs.txt", header = FALSE, 
                       col.names = c("type", "calories", "sodium"),
                       colClasses = c("factor","NULL","numeric"))


# Display structure of hotdogs2
str(hotdogs2)
```

## readr
**默认了`header = TRUE`,`stringsAsfactor = FALSE`**

`readr`这个包读入的文档都会成为一种 **tibble** 格式的 `dataframe`，这里不仅会给出每列的数据，也不会将`chr`转成`factor`.

`read_delim` (**注意不是上面的 read.delim()**)是上级命令，根据参数的改变可以变成 `read_csv()`和 `read_tsv()`.

    Where you use read_csv() to easily read in CSV files, you use read_tsv() to easily read in TSV files. 
    TSV is short for tab-separated values.

### read_tsv()
```r
# readr is already loaded

# Column names
properties <- c("area", "temp", "size", "storage", "method",
                "texture", "flavor", "moistness")

# Import potatoes.txt: potatoes
potatoes<- read_tsv("potatoes.txt",
                    col_names = properties) # 因为这个txt没有列名，所以在读入时添加一个

# Call head() on potatoes
head(potatoes)
```

### read_delim()

```r
# readr is already loaded

# Column names
properties <- c("area", "temp", "size", "storage", "method",
                "texture", "flavor", "moistness")

# Import potatoes.txt using read_delim(): potatoes
potatoes<- read_delim("potatoes.txt",
            delim= "\t",
            col_names = properties)

# Print out potatoes
potatoes
```

### 参数：skip 和 n_max


 - `skip` specifies the **number of lines** you're ignoring in the flat file before actually starting to import data.
 - `n_max` specifies the number of lines you're actually importing.

比方说你有一个20行的`CSV`文件，当你设置 `skip = 2 and n_max = 3`，也就是说你跳过的前**两行**，读取了**3,4,5，三行**

**Watch out**: Once you skip some lines, you also skip the **first line** that can contain column names第一行的列名也会被跳过；

```r
# Column names
properties <- c("area", "temp", "size", "storage", "method",
                "texture", "flavor", "moistness")

# Import 5 observations from potatoes.txt: potatoes_fragment
potatoes_fragment <- read_tsv("potatoes.txt", 
                     skip = 6, 
                     n_max = 5, 
                     col_names = properties) # 因为第一行列名被skip了，所以要补上
```

### 参数：col_types

类型缩写：
 - `c` character
 - `d` double
 - `i` integer
 - `l` logical
 - `_` skips the column as a whole.
 
```r
# Import all data, but force all columns to be character: potatoes_char
potatoes_char <- read_tsv("potatoes.txt", 
                           col_types = "cccccccc", 
                           col_names = properties)

# Print out structure of potatoes_char
str(potatoes_char)
```

### Collector 
当然也可以用另外指定的方法来对col_types进行定义。

```r
# The collectors you will need to import the data
fac <- col_factor(levels = c("Beef", "Meat", "Poultry"))
int <- col_integer()

# Edit the col_types argument to import the data correctly: hotdogs_factor
hotdogs_factor <- read_tsv("hotdogs.txt",
                           col_names = c("type", "calories", "sodium"),
                           col_types = list(fac, int, int))
```


## data.table

### fread(）

similar to `read.table()`

Now that you know the basics about` fread()`

You should know about two arguments of the function: `drop` and `select`, to **drop** or **select** variables of interest.

假设你现在有一个5 列的表格，你想要保留第1和第5行，名字分别是 `a`,`e`.那么你就可以用以下语法：
```r
fread("path/to/file.txt", drop = 2:4) # 用位置索引 drop掉 2：4列
fread("path/to/file.txt", select = c(1, 5)) # 用位置索引 保留1和5列
fread("path/to/file.txt", drop = c("b", "c", "d")) # 用名称drop
fread("path/to/file.txt", select = c("a", "e")) # 用名称保留
```
```r
# Import columns 6 and 8 of potatoes.csv: potatoes
potatoes<- fread("potatoes.csv",
                 select= c(6,8))# 只保留第6，8列

# Plot texture (x) and moistness (y) of potatoes
plot(potatoes)
```

## readxl

 - `excel_sheet()`: list different sheets in excel
 - `read_excel()`: actually import data into R
```r
read_excel("cities.xlsx") # 读取excel时，默认读取第一个sheet
read_excel("cities.xlsx", sheet = 2) # 可以通过数字和sheet名称来读取你想要的那张sheet
read_excel("cities.xlsx", sheet = "year_2000")
```

```r
# Load the readxl package
library(readxl)

# Print the names of all worksheets
excel_sheets("urbanpop.xlsx")

# Read the sheets, one by one
pop_1 <- read_excel("urbanpop.xlsx", sheet = 1)
pop_2 <- read_excel("urbanpop.xlsx", sheet = 2)
pop_3 <- read_excel("urbanpop.xlsx", sheet = 3)

# Put pop_1, pop_2 and pop_3 in a list: pop_list
pop_list<- list(pop_1,pop_2,pop_3)

# Display the structure of pop_list
str(pop_list)
summary(pop_list)
```
上面的pop_list建立的方法可以用`lapply`来取代.

```r
# Read all Excel sheets with lapply(): pop_list
pop_list<- lapply(excel_sheets("urbanpop.xlsx"), #循环处理的对象
                  read_excel,       # 处理的方法
                  path = "urbanpop.xlsx") # 这个read_excel里的路径
                                  # 相当于 read_excel("urbanpop.xlsx", sheet = excel_sheets("urbanpop.xlsx"))
                  

# Display the structure of pop_list
str(pop_list)
```

## readxl + 其他参数

```r
read_excel(path, sheet = 1,            
           col_names = TRUE,  # 首行为列名          
           col_types = NULL,            
           skip = 0)

read_excel("cities.xlsx", 
           col_types = c("text", "text")) # 设定列的类型

read_excel("cities.xlsx",             
           col_types = c("text", "blank")) # 当列的类型设定为 blank 时，证明你忽略了这一列，即不读入这一列

read_excel("cities.xlsx",             
           col_names = c("Capital", "Population"),            
           skip = 2) # 跳过前两行
#=====================================================================================
# Import the first Excel sheet of urbanpop_nonames.xlsx (R gives names): pop_a
pop_a<- read_excel("urbanpop_nonames.xlsx",
                    sheet = 1,
                    col_names = TRUE)

# Import the first Excel sheet of urbanpop_nonames.xlsx (specify col_names): pop_b
cols <- c("country", paste0("year_", 1960:1966))
pop_b<- read_excel("urbanpop_nonames.xlsx",
                    sheet = 1,
                    col_names = cols)

# Print the summary of pop_a
summary(pop_a)

# Print the summary of pop_b
summary(pop_b)
```

## gdata (package)


![](https://raw.githubusercontent.com/w1nsan/img_repo/master/gdata.PNG)


To import data from excel ,`gdata` use Perl.

如果要读取比较大的 EXCEL，还是用`readr`比较好。因为gdata有一个转为 CSV 的过程

```r
# 加载包
library(gdata)

# 读取第二张数据表
urban_pop<- read.xls("urbanpop.xls",sheet = 2)

# 打印前11行观测值
head(urban_pop,11)
```

根据上图所示，`read.xls()`的本质是先用 Perl 语言将 excel 转换为 CSV，然后再执行`read.csv()`.

因此能用来定义`read.csv()`的参数**都可以**用来定义`read.xls()`

```r
# 设定新的列名
columns <- c("country", paste0("year_", 1967:1974))

# 读取第二张表
urban_pop <- read.xls("urbanpop.xls", sheet = 2,
                      skip = 50,  # 跳过前50行
                      header = FALSE, # 因为第一行也被忽略掉所以这里不能以第一行为表头
                      stringsAsFactors = FALSE, # 不能读取为因子
                      col.names = columns) #赋予列名

# Print first 10 observation of urban_pop
head(urban_pop,10)
```

## 对读取到的xls数据作处理

```r
# Add code to import data from all three sheets in urbanpop.xls
path <- "urbanpop.xls"
urban_sheet1 <- read.xls(path, sheet = 1, stringsAsFactors = FALSE)
urban_sheet2 <- read.xls(path, sheet = 2, stringsAsFactors = FALSE)
urban_sheet3 <- read.xls(path, sheet = 3, stringsAsFactors = FALSE)

# Extend the cbind() call to include urban_sheet3: urban
urban <- cbind(urban_sheet1, urban_sheet2[-1],urban_sheet3[-1])

# Remove all rows with NAs from urban: urban_clean
urban_clean<- na.omit(urban)

# Print out a summary of urban_clean
summary(urban_clean)
```

## XLConnet (package)

Working `excel` through `R` !!

 - 先把数据用`loadWorkbook()`读取到XLConnet里，一般用`book`表示；
 
 - 再通过不同的函数来读取`book`里的数据

```r
# Load the XLConnect package
library(XLConnect)

# 把excel表读取到book里（这个book是特殊的格式）
my_book<- loadWorkbook("urbanpop.xlsx")

# my_book的数据类型是“XLConnet"，需要通过其他函数里提取里面的数据
class(my_book)

[1] "workbook"
attr(,"package")
[1] "XLConnect"
```

用不同的函数读取 book里面的数据；
```r
# 加载 book
my_book <- loadWorkbook("urbanpop.xlsx")

# 获取文件的列表
getSheets(my_book)

# 读取 book 里的数据
readWorksheet(my_book,
              sheet = 2,)
```

进一步提取行和列
```r
# Build connection to urbanpop.xlsx
my_book <- loadWorkbook("urbanpop.xlsx")

# Import columns 3, 4, and 5 from second sheet in my_book: urbanpop_sel
urbanpop_sel <- readWorksheet(my_book, 
                              sheet = 2,
                              startCol = 3,
                              endCol = 5)

# Import first column from second sheet in my_book: countries
countries<- readWorksheet(my_book,
                          sheet = 2,
                          endCol = 1)

# cbind() urbanpop_sel and countries together: selection
selection<- cbind(countries,urbanpop_sel)
```

## 对Excel列表的更改

### 包括 添加/重命名/删除

在R 里增加一张EXCEL的列表需要经过 `creatsheet()`到`writeWorksheet()`最后要`saveWorksheet()`才能在EXCEL文件里看见新增的内容
![](https://raw.githubusercontent.com/w1nsan/img_repo/master/xlconnect.PNG)

### Add worksheet

Where readxl and gdata were only able to import Excel data, XLConnect's approach of providing an actual interface to an Excel file makes it able to edit your Excel files from inside R. In this exercise, you'll **create a new sheet**. In the next exercise, you'll populate the sheet with data, and save the results in a new Excel file.

You'll continue to work with `urbanpop.xlsx`. The `my_book `object that links to this Excel file is already available.

```r
# Build connection to urbanpop.xlsx
my_book <- loadWorkbook("urbanpop.xlsx")

# Add a worksheet to my_book, named "data_summary"
createSheet(my_book, name = "data_summary")

# Use getSheets() on my_book
getSheets(my_book)
```

### Populate worksheet 填充数据

The first step of creating a sheet is done; let's populate it with some data now! 

`summ`, a data frame with some summary statistics on the two Excel sheets is already coded so you can take it from there.

```r
# 加载excel文件到workbook
my_book <- loadWorkbook("urbanpop.xlsx")

# 新建一张sheet，名字是 "data_summary"（没有内容的）
createSheet(my_book, "data_summary")

# 创建一个df： summ
sheets <- getSheets(my_book)[1:3] # 得到workbook文件里三个表格的名称
dims <- sapply(sheets, 
              function(x) dim(readWorksheet(my_book, sheet = x)), # 获得每张表格的行列数目
              USE.NAMES = FALSE)

summ <- data.frame(sheets = sheets, # 第一列是表格名称
                   nrows = dims[1, ], #第二列是三张表格的行数
                   ncols = dims[2, ]) #第三列是三张表格的列数

# Add data in summ to "data_summary" sheet
writeWorksheet(my_book,  # 被加入的对象
                summ,    # 加入的内容
                sheet = "data_summary") # 加入的位置，用名称或数字来指代都可以

# Save workbook as summary.xlsx
saveWorkbook(my_book,file="summary.xlsx")
```

### Renaming sheets 重命名

Come to think of it, `"data_summary"` is not an ideal name. As the summary of these excel sheets is always data-related, you simply want to name the sheet `"summary"`.

The workspace already contains a workbook, `my_book`, that refers to an Excel file with 4 sheets: the three data sheets, and the `"data_summary"` sheet.

先查看一下函数的语法：
```r
> args(renameSheet)
function (object, sheet, newName) 

```

```r
# 改名 "data_summary" sheet to "summary"
renameSheet(my_book,
            sheet = "data_summary",# present name
            newName= "summary") # new name

getSheets(my_book) # 查看更改情况

# 另存为
saveWorkbook(my_book, file = "renamed.xlsx")
```

### Removing sheets

After presenting the new Excel sheet to your peers, it appears not everybody is a big fan. Why summarize sheets and store the info in Excel if all the information is implicitly available? To hell with it, just remove the entire fourth sheet!

照例先查看相关函数 `removeSheets()`的参数设定：

```r
> args(removeSheet)
function (object, sheet) 
NULL
```

删除上一个练习中新建的表格`summary`
```r
# Load the XLConnect package
library(XLConnect)

# Build connection to renamed.xlsx: my_book
my_book<- loadWorkbook("renamed.xlsx")

# Remove the fourth sheet
removeSheet(my_book, "summary")
getSheets(my_book)
# Save workbook to "clean.xlsx"
saveWorkbook(my_book, file = "clean.xlsx")
```